# **Table Design**
## RAW TABLE


In [54]:
import pandas as pd
raw_table = pd.DataFrame([['DATETIME', 'VARCHAR2(30 BYTE)', 'N', 'Yes', 'null', 1, 'Data from raw file'],
                          ['VIDEOTITLE', 'VARCHAR2(200 BYTE)', 'N', 'Yes', 'null', 2, 'Data from raw file'],
                          ['EVENTS', 'VARCHAR2(150 BYTE)', 'N', 'Yes', 'null', 3, 'Data from raw file']],
                  columns=['COLUMN_NAME', 'DATA_TYPE', 'PK', 'NULLABLE', 'DATA_DEFAULT', 'COLUMN_ID', 'COMMENTS'])
raw_table.style.hide_index()

COLUMN_NAME,DATA_TYPE,PK,NULLABLE,DATA_DEFAULT,COLUMN_ID,COMMENTS
DATETIME,VARCHAR2(30 BYTE),N,Yes,null,1,Data from raw file
VIDEOTITLE,VARCHAR2(200 BYTE),N,Yes,null,2,Data from raw file
EVENTS,VARCHAR2(150 BYTE),N,Yes,null,3,Data from raw file


## Dimension Table DIMDATE 

In [53]:
import pandas as pd

dimdate = pd.DataFrame([['DATETIME_SKEY', 'TIMESTAMP', 'Y', 'No', 'NOT NULL', 1, 'Data derived from DIMDATE.DATETIME_SKEY']],
                  columns=['COLUMN_NAME', 'DATA_TYPE', 'PK', 'NULLABLE', 'DATA_DEFAULT', 'COLUMN_ID', 'COMMENTS'])

dimdate.style.hide_index()

COLUMN_NAME,DATA_TYPE,PK,NULLABLE,DATA_DEFAULT,COLUMN_ID,COMMENTS
DATETIME_SKEY,TIMESTAMP,Y,No,NOT NULL,1,Data derived from DIMDATE.DATETIME_SKEY


## Dimension Table DIMPLATFORM

In [52]:
import pandas as pd

dimplatform = pd.DataFrame([['PLATFORM_SKEY', 'INTEGER', 'Y', 'No', 'NOT NULL', 1, 'Data derived DIMPLATFORM.PLATFORM_SKEY'],
                          ['PLATFORM_TYPE', 'VARCHAR2(200 BYTE)', 'N', 'No', 'NOT NULL', 2, 'Data derived DIMPLATFORM.PLATFORM']],
                  columns=['COLUMN_NAME', 'DATA_TYPE', 'PK', 'NULLABLE', 'DATA_DEFAULT', 'COLUMN_ID', 'COMMENTS'])
                  
dimplatform.style.hide_index()

COLUMN_NAME,DATA_TYPE,PK,NULLABLE,DATA_DEFAULT,COLUMN_ID,COMMENTS
PLATFORM_SKEY,INTEGER,Y,No,NOT NULL,1,Data derived DIMPLATFORM.PLATFORM_SKEY
PLATFORM_TYPE,VARCHAR2(200 BYTE),N,No,NOT NULL,2,Data derived DIMPLATFORM.PLATFORM


## Dimension Table DIMSITE


In [55]:
import pandas as pd

dimsite = pd.DataFrame([['SITE_SKEY', 'INTEGER', 'Y', 'No', 'NOT NULL', 1, 'Data derived DIMSITE.SITE_SKEY'],
                          ['SITE', 'VARCHAR(200)', 'N', 'No', 'NOT NULL', 2, 'Data derived DIMSITE.SITE']],
                  columns=['COLUMN_NAME', 'DATA_TYPE', 'PK', 'NULLABLE', 'DATA_DEFAULT', 'COLUMN_ID', 'COMMENTS'])
                  
dimsite.style.hide_index()

COLUMN_NAME,DATA_TYPE,PK,NULLABLE,DATA_DEFAULT,COLUMN_ID,COMMENTS
SITE_SKEY,INTEGER,Y,No,NOT NULL,1,Data derived DIMSITE.SITE_SKEY
SITE,VARCHAR(200),N,No,NOT NULL,2,Data derived DIMSITE.SITE


## Dimension Table DIMVIDEO


In [56]:
import pandas as pd

dimvideo = pd.DataFrame([['VIDEO_SKEY', 'INTEGER', 'Y', 'No', 'NOT NULL', 1, 'Data derived from DIMVIDEO.VIDEO_SKEY'],
                          ['VIDEO_TITLE', 'TEXT', 'N', 'No', 'NOT NULL', 2, 'Data derived from DIMVIDEO.VIDEO_TITLE']],
                  columns=['COLUMN_NAME', 'DATA_TYPE', 'PK', 'NULLABLE', 'DATA_DEFAULT', 'COLUMN_ID', 'COMMENTS'])
                  
dimvideo.style.hide_index()

COLUMN_NAME,DATA_TYPE,PK,NULLABLE,DATA_DEFAULT,COLUMN_ID,COMMENTS
VIDEO_SKEY,INTEGER,Y,No,NOT NULL,1,Data derived from DIMVIDEO.VIDEO_SKEY
VIDEO_TITLE,TEXT,N,No,NOT NULL,2,Data derived from DIMVIDEO.VIDEO_TITLE


## Fact Table FACTTABLE


In [57]:
import pandas as pd
facttable = pd.DataFrame([['DATETIME_SKEY', 'TIMESTAMP', 'N', 'No', 'NOT NULL', 1, 'Data derived from FACTTABLE.DATETIME_SKEY'],
                          ['Platform_SKEY', 'INTEGER', 'N', 'No', 'NOT NULL', 2, 'Data derived from FACTTABLE.PLATFORM_SKEY'],
                          ['Site_SKEY', 'INTEGER', 'N', 'No', 'NOT NULL', 3, 'Data derived from FACTTABLE.SITE_SKEY'],
                          ['Video_SKEY', 'INTEGER', 'N', 'No', 'NOT NULL', 4, 'Data derived from FACTTABLE.VIDEO_SKEY'],
                          ['Events', 'VARCHAR2(150 BYTE)', 'N', 'No', 'NOT NULL', 5, 'Data derived from FACTTABLE.EVENTS']],
                  columns=['COLUMN_NAME', 'DATA_TYPE', 'PK', 'NULLABLE', 'DATA_DEFAULT', 'COLUMN_ID', 'COMMENTS'])
                  
facttable.style.hide_index()

COLUMN_NAME,DATA_TYPE,PK,NULLABLE,DATA_DEFAULT,COLUMN_ID,COMMENTS
DATETIME_SKEY,TIMESTAMP,N,No,NOT NULL,1,Data derived from FACTTABLE.DATETIME_SKEY
Platform_SKEY,INTEGER,N,No,NOT NULL,2,Data derived from FACTTABLE.PLATFORM_SKEY
Site_SKEY,INTEGER,N,No,NOT NULL,3,Data derived from FACTTABLE.SITE_SKEY
Video_SKEY,INTEGER,N,No,NOT NULL,4,Data derived from FACTTABLE.VIDEO_SKEY
Events,VARCHAR2(150 BYTE),N,No,NOT NULL,5,Data derived from FACTTABLE.EVENTS


# **Pre-process the Raw Data**

## 1. Data auditing

In [117]:
import pandas as pd

df = pd.read_csv('source.csv', usecols=[0,1,2])
df.columns = ['DateTime', 'VideoTitle', 'events']

print(pd.Series({c: df[c].map(lambda x: len(str(x))).max() for c in df}))

DateTime       24
VideoTitle    157
events         95
dtype: int64


## 2. Identify Objects for ETL Process

In [121]:
# Identify the type of PLATFORM and SITE
import pandas as pd

df = pd.read_csv('source.csv', usecols=[0,1,2])
df.columns = ['DateTime', 'VideoTitle', 'events']
df[['0','1','2','3','4']] = df['VideoTitle'].str.split("|",expand=True,)

df['0'].unique()

array(['App Web', 'news', 'App iPhone', 'App Android', 'App iPad'],
      dtype=object)

## 3. The sql script to create the table

check <font color='#33AFFF'>snowflake.sql</font>

# **NOTE**

1. SKEY stands for surrogate key.  
2. The current design is <font color='#33AFFF'>Dimension Type One</font>.  
3. If the source dimension data contains not only the PK but also some attributes, and we want to track the changes of attributes, we should use <font color='#33AFFF'>Dimension Type Two</font>. 

### **One sample of Dimension Type Two**

In [207]:
import pandas as pd
sample_table = pd.DataFrame([['P01', 'iPhone 13', '1399', 'Noel Leeming'],['P03', 'iPhone 13 Pro Max', '1999', 'Noel Leeming']],columns=['Product_ID', 'Product', 'Price', 'Location'])
sample_table.style.hide_index().set_caption('Data from current date 04/03/2022')

Product_ID,Product,Price,Location
P01,iPhone 13,1399,Noel Leeming
P03,iPhone 13 Pro Max,1999,Noel Leeming


### **Data in dimension table**

In [204]:
import pandas as pd

dim_table = pd.DataFrame([['1', 'P01', 'iPhone 13', '1499', 'Noel Leeming', 'Y', '31/12/2021', '31/12/9999'], ['2', 'P02', 'iPhone 13 Pro', '1799', 'Noel Leeming', 'Y', '20/01/2022', '31/12/9999']], columns=['Product_SKEY', 'Product_ID', 'Product', 'Price', 'Location','Current_Flag', 'Start_Date', 'End_Date'])
dim_table.style.hide_index().set_caption('Data in dimension table')

Product_SKEY,Product_ID,Product,Price,Location,Current_Flag,Start_Date,End_Date
1,P01,iPhone 13,1499,Noel Leeming,Y,31/12/2021,31/12/9999
2,P02,iPhone 13 Pro,1799,Noel Leeming,Y,20/01/2022,31/12/9999


### Update data in dimension table:
* new product **(P03)**  
* update product **(P01)**

In [205]:
import pandas as pd
dim_table = pd.DataFrame([['1', 'P01', 'iPhone 13', '1499', 'Noel Leeming', 'N', '31/12/2021', '03/03/2022'],
                          ['2', 'P02', 'iPhone 13 Pro', '1799', 'Noel Leeming', 'Y', '20/01/2022', '31/12/9999'],
                          ['3', 'P03', 'iPhone 13 Pro Max', '1999', 'Noel Leeming', 'Y', '04/03/2022', '31/12/9999'],
                          ['4', 'P01', 'iPhone 13', '1399', 'Noel Leeming', 'Y', '04/03/2022', '31/12/9999']],
                  columns=['Product_SKEY', 'Product_ID', 'Product', 'Price', 'Location','Current_Flag', 'Start_Date', 'End_Date'])
    
dim_table.style.hide_index().set_caption('Update dimension table')

Product_SKEY,Product_ID,Product,Price,Location,Current_Flag,Start_Date,End_Date
1,P01,iPhone 13,1499,Noel Leeming,N,31/12/2021,03/03/2022
2,P02,iPhone 13 Pro,1799,Noel Leeming,Y,20/01/2022,31/12/9999
3,P03,iPhone 13 Pro Max,1999,Noel Leeming,Y,04/03/2022,31/12/9999
4,P01,iPhone 13,1399,Noel Leeming,Y,04/03/2022,31/12/9999


When there is a **new record** coming in, we generate a new record with new SKEY, 
* Current_Flag = ‘Y’
* Start_Date = Current_Date
* End_Date = 31/12/9999

When there is a **updated record** coming in, we also generate a new record with new SKEY,
* Current_Flag = ‘Y’
* Start_Date = Current_Date
* End_Date = 31/12/9999

And at same time we need to update the **old record** in dimension table with 
* Current_Flag = ‘N’
* End_Date = Current_Date – 1

Therefore, when we populate new records into fact table, we need to put a filter such as **Current_Flag = ‘Y’** in order to get the correct SKEY; 

If we want to track the history data in dimension table for **certain days or certain period**, we need to put a time range filter such as **EVENT_DATE(or CONTACT_DATE)** between Start_Date and End_Date

For example, if in fact table we see a transaction like customer purchased product(P01) on 01/02/22, by looking at product dimension table, we could find the price that customer paid at that moment was 1499 not 1399, although 1399 is the current price of P01.